In [1]:
import joblib
import os

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from src.utils import encode_text

In [11]:
import torch
torch.__version__

'1.8.0'

## Load Data, Encode Text

In [2]:
full_df = pd.read_csv('data/raw_combined.csv')
full_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,all,pct_punctuation
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,0,0.041667
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,0,0.107143
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,0,0.025751
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,0,0.040193
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,0,0.074627


In [3]:
vocab, encoded_text = encode_text(
    full_df, 
    use_cache=False,
    max_length=500,
    vocab_length=10000
)

In [4]:
# should be (n comments, max_length + 1)
encoded_text.shape

(223549, 501)

In [6]:
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat',
                 'insult', 'identity_hate']
labels = full_df[label_columns].values

full_data = np.concatenate([labels, encoded_text], axis=1)

## Train / Val / Test Split

In [14]:
data_dir = 'processed/'
# save vocab to each so it's available
for subdir in ['train', 'val', 'test']:
    directory = os.path.join(data_dir, subdir)
    os.makedirs(directory, exist_ok=True)
    with open(os.path.join(directory, 'vocab.joblib'), 'wb') as f:
        joblib.dump(vocab, f)

In [15]:


# split out test first
train, test = train_test_split(
    full_data, 
    test_size=0.1, 
    random_state=1
)

with open(os.path.join(data_dir, 'test/data.joblib'), 'wb') as f:
    joblib.dump(test, f)

train, val = train_test_split(
    train,
    test_size=0.2,
    random_state=1
)


with open(os.path.join(data_dir, 'train/data.joblib'), 'wb') as f:
    joblib.dump(train, f)
with open(os.path.join(data_dir, 'val/data.joblib'), 'wb') as f:
    joblib.dump(val, f)

print(train.shape, val.shape, test.shape)

(160955, 507) (40239, 507) (22355, 507)
